In [1]:
import pandas as pd
import os
from pathlib import Path
from datetime import datetime
import pyarrow.parquet as pq
import pyarrow as pa
from typing import List, Optional

# Check if running in Databricks
IS_DATABRICKS = "DATABRICKS_RUNTIME_VERSION" in os.environ

if IS_DATABRICKS:
    from pyspark.sql import SparkSession
    import pyspark.sql.functions as F
def load_fred_data(fred_dir: str, is_databricks: bool = False, delta_path: str = None) -> pd.DataFrame:
    try:
        if is_databricks:
            spark = SparkSession.builder.appName("FREDDataLoader").getOrCreate()
            spark_df = spark.read.format("delta").load(delta_path)
            df = spark_df.toPandas()
            print(f"✅ Loaded FRED data from Delta Lake at {delta_path}")
        else:
            fred_dir = Path(fred_dir)
            dfs = []
            for file in fred_dir.glob("*.csv"):
                series_id = file.stem
                df = pd.read_csv(file)
                df['date'] = pd.to_datetime(df['date'])
                df.set_index('date', inplace=True)
                df = df[[series_id]]
                dfs.append(df)
            if not dfs:
                raise ValueError(f"No CSV files found in {fred_dir}")
            df = pd.concat(dfs, axis=1, join="outer")
            print(f"✅ Loaded {len(dfs)} FRED series from {fred_dir}")
        return df
    except Exception as e:
        print(f"❌ Error loading FRED data: {str(e)}")
        raise

def load_yfinance_data(ticker: str, yf_dir: str) -> pd.DataFrame:
    try:
        file_path = Path(yf_dir) / f"{ticker.upper()}_monthly.csv"
        if not file_path.exists():
            raise FileNotFoundError(f"No data file found for {ticker} at {file_path}")
        df = pd.read_csv(file_path)
        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)
        df = df[['Close']].rename(columns={'Close': f'{ticker}_Close'})
        print(f"✅ Loaded Yahoo Finance data for {ticker} from {file_path}")
        return df
    except Exception as e:
        print(f"❌ Error loading Yahoo Finance data for {ticker}: {str(e)}")
        raise

def engineer_features(df: pd.DataFrame, target_col: str, lag_periods: List[int] = [1, 3, 6], ma_windows: List[int] = [3, 12]) -> pd.DataFrame:
    try:
        print("🛠️ Engineering features: lags and moving averages")
        df = df.copy()
        for col in df.columns:
            for lag in lag_periods:
                df[f'{col}_lag{lag}'] = df[col].shift(lag)
        for window in ma_windows:
            df[f'{target_col}_ma{window}'] = df[target_col].rolling(window=window).mean()
        df = df.dropna()
        print(f"✅ Feature engineering complete, resulting shape: {df.shape}")
        return df
    except Exception as e:
        print(f"❌ Error in feature engineering: {str(e)}")
        raise
def save_to_storage(df: pd.DataFrame, output_path: str, ticker: str, is_databricks: bool = False) -> None:
    try:
        output_file = f"{output_path}/{ticker}_combined.parquet"
        print(f"💾 Saving data for {ticker} to {output_file}")

        if is_databricks:
            spark = SparkSession.builder.appName("CombinedDataSaver").getOrCreate()
            spark_df = spark.createDataFrame(df.reset_index())
            spark_df.write.format("delta").mode("overwrite").save(output_file)
        else:
            os.makedirs(os.path.dirname(output_file), exist_ok=True)
            table = pa.Table.from_pandas(df)
            pq.write_table(table, output_file)

        print(f"✅ Data saved successfully for {ticker}")
    except Exception as e:
        print(f"❌ Error saving data for {ticker}: {str(e)}")
        raise
        
        


In [38]:
fred_dir = Path(fred_dir)
dfs = []
for file in fred_dir.glob("*.csv"):
    series_id = file.stem
    df = pd.read_csv(file)
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    df = df[[series_id]]  # Keep only the series column
    dfs.append(df)
if not dfs:
    raise ValueError(f"No CSV files found in {fred_dir}")
df = pd.concat(dfs, axis=1, join="outer")



In [31]:
df = pd.read_csv("C:/Users/Steel/Desktop/Projects/intel-sweep/intel-sweep/src/data/fred_economic_indicators3/BAA.csv")
df2 =pd.read_csv("C:/Users/Steel/Desktop/Projects/intel-sweep/intel-sweep/src/data/fred_economic_indicators3/T10Y2Y.csv")
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df2['date'] = pd.to_datetime(df2['date'])
df2.set_index('date', inplace=True)

In [29]:
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df2['date'] = pd.to_datetime(df2['date'])
df2.set_index('date', inplace=True)

In [33]:
df3 = [df,df2]

In [35]:
df = pd.concat(df3, axis=1, join="outer")

In [39]:
df

,BAA,BOPGSTB,BUSINV,CES0500000003,CFNAI,CIVPART,CPIAUCSL,CSUSHPINSA,DCOILWTICO,DGORDER,...,SP500,SRVPRD,T10Y2Y,TB3MS,TCU,TEDRATE,UMCSENT,UNRATE,USSLIND,VIXCLS
date,,,,,,,,,,,,,,,,,,,,,
2000-01-01,8.33,-27271.888889,1.142746e+06,NaN,0.362903,67.400000,169.250000,100.000000,27.259474,204411.000000,...,NaN,105853.789474,NaN,5.32,82.079592,0.718947,112.0,4.00,1.640000,NaN
2000-02-01,8.29,-29398.111111,1.148697e+06,NaN,-0.267059,67.433333,170.033333,100.570500,29.366000,190101.095238,...,NaN,106017.235294,NaN,5.55,82.078657,0.551500,111.3,4.10,1.491045,NaN
2000-03-01,8.37,-31156.941176,1.152465e+06,NaN,0.549130,67.300000,171.175000,101.466429,29.759167,198210.571429,...,NaN,106500.000000,NaN,5.69,82.164429,0.502917,107.1,4.05,1.717385,NaN
2000-04-01,8.40,-29278.722222,1.159025e+06,NaN,0.613462,67.400000,171.100000,102.540500,25.722105,197263.142857,...,NaN,106755.222222,NaN,5.66,82.469843,0.650000,109.2,3.90,1.736308,NaN
2000-05-01,8.90,-30083.611111,1.165796e+06,NaN,-0.556970,67.133333,171.325000,103.701571,28.788182,196322.476190,...,NaN,106974.375000,NaN,5.79,82.450418,0.959048,110.7,4.05,1.526508,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-01,6.08,-131017.000000,2.588530e+06,35.85,-0.123333,62.600000,319.086000,323.570000,75.742500,286343.000000,...,5979.515500,137398.000000,0.357143,4.21,77.704633,NaN,71.7,4.00,NaN,16.763636
2025-02-01,5.92,-122662.000000,2.589957e+06,35.92,0.210000,62.400000,319.775000,324.923000,71.533158,289296.666667,...,6038.690000,137500.000000,0.241053,4.22,78.192950,NaN,64.7,4.10,NaN,16.968000
2025-03-01,5.93,NaN,NaN,36.00,-0.030000,62.500000,319.615000,NaN,68.239048,315729.000000,...,5683.983333,137698.000000,0.310476,4.20,77.849100,NaN,57.0,4.20,NaN,21.841429


In [2]:
fred_dir = 'C:/Users/Steel/Desktop/Projects/intel-sweep/intel-sweep/src/data/fred_economic_indicators3'
yf_dir = 'C:/Users/Steel/Desktop/Projects/intel-sweep/intel-sweep/src/data/yf_monthly'
output_path = 'C:/Users/Steel/Desktop/Projects/intel-sweep/intel-sweep/data/combined'
tickers = ['AAPL']
delta_path = '/mnt/fred_data'  # Only used if IS_DATABRICKS = True

# Run the pipeline
fred_data = load_fred_data(fred_dir, IS_DATABRICKS, delta_path)

fred_data = fred_data.fillna(method='ffill').fillna(method='bfill')

✅ Loaded 36 FRED series from C:\Users\Steel\Desktop\Projects\intel-sweep\intel-sweep\src\data\fred_economic_indicators3


In [3]:
fred_data

,BAA,BOPGSTB,BUSINV,CES0500000003,CFNAI,CIVPART,CPIAUCSL,CSUSHPINSA,DCOILWTICO,DGORDER,...,SP500,SRVPRD,T10Y2Y,TB3MS,TCU,TEDRATE,UMCSENT,UNRATE,USSLIND,VIXCLS
date,,,,,,,,,,,,,,,,,,,,,
2000-01-01,8.33,-27271.888889,1.142746e+06,NaN,0.362903,67.400000,169.250000,100.000000,27.259474,204411.000000,...,NaN,105853.789474,NaN,5.32,82.079592,0.718947,112.0,4.00,1.640000,NaN
2000-02-01,8.29,-29398.111111,1.148697e+06,NaN,-0.267059,67.433333,170.033333,100.570500,29.366000,190101.095238,...,NaN,106017.235294,NaN,5.55,82.078657,0.551500,111.3,4.10,1.491045,NaN
2000-03-01,8.37,-31156.941176,1.152465e+06,NaN,0.549130,67.300000,171.175000,101.466429,29.759167,198210.571429,...,NaN,106500.000000,NaN,5.69,82.164429,0.502917,107.1,4.05,1.717385,NaN
2000-04-01,8.40,-29278.722222,1.159025e+06,NaN,0.613462,67.400000,171.100000,102.540500,25.722105,197263.142857,...,NaN,106755.222222,NaN,5.66,82.469843,0.650000,109.2,3.90,1.736308,NaN
2000-05-01,8.90,-30083.611111,1.165796e+06,NaN,-0.556970,67.133333,171.325000,103.701571,28.788182,196322.476190,...,NaN,106974.375000,NaN,5.79,82.450418,0.959048,110.7,4.05,1.526508,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-01,6.08,-131017.000000,2.588530e+06,35.85,-0.123333,62.600000,319.086000,323.570000,75.742500,286343.000000,...,5979.515500,137398.000000,0.357143,4.21,77.704633,NaN,71.7,4.00,NaN,16.763636
2025-02-01,5.92,-122662.000000,2.589957e+06,35.92,0.210000,62.400000,319.775000,324.923000,71.533158,289296.666667,...,6038.690000,137500.000000,0.241053,4.22,78.192950,NaN,64.7,4.10,NaN,16.968000
2025-03-01,5.93,NaN,NaN,36.00,-0.030000,62.500000,319.615000,NaN,68.239048,315729.000000,...,5683.983333,137698.000000,0.310476,4.20,77.849100,NaN,57.0,4.20,NaN,21.841429


In [8]:
stock_data = load_yfinance_data('AAPL', yf_dir)

✅ Loaded Yahoo Finance data for AAPL from C:\Users\Steel\Desktop\Projects\intel-sweep\intel-sweep\src\data\yf_monthly\AAPL_monthly.csv


In [9]:
stock_data

,AAPL_Close
Date,
NaT,AAPL
1985-01-01,0.0995
1985-02-01,0.0849
1985-03-01,0.0759
1985-04-01,0.0729
...,...
2025-01-01,235.4321
2025-02-01,241.258
2025-03-01,221.8391


In [10]:
 combined = fred_data.join(stock_data, how='inner').dropna()

In [11]:
combined

,BAA,BOPGSTB,BUSINV,CES0500000003,CFNAI,CIVPART,CPIAUCSL,CSUSHPINSA,DCOILWTICO,DGORDER,...,SRVPRD,T10Y2Y,TB3MS,TCU,TEDRATE,UMCSENT,UNRATE,USSLIND,VIXCLS,AAPL_Close
2017-07-01,4.39,-43842.454545,1.878008e+06,26.358571,-0.142941,62.866667,244.254167,193.660364,46.600000,223990.454545,...,126647.500000,0.945238,1.07,76.574427,0.240476,93.4,4.300000,1.297000,10.264500,34.6436
2017-10-01,4.32,-45681.700000,1.892606e+06,26.497143,0.698529,62.700000,246.610333,195.238733,51.577727,233244.818182,...,126950.727273,0.810476,1.07,77.306918,0.286190,100.7,4.150000,1.577826,10.125455,39.529
2018-01-01,4.26,-51366.000000,1.919995e+06,26.724000,-0.245758,62.700000,248.878000,196.254524,63.698571,233560.727273,...,127365.888889,0.551905,1.41,77.508650,0.320000,95.7,4.050000,1.554737,11.062381,39.2933
2018-04-01,4.67,-46001.636364,1.935419e+06,26.890000,0.379211,62.850000,250.095667,200.750930,66.253810,245270.800000,...,127865.111111,0.484762,1.76,78.814927,0.591500,98.8,3.950000,1.568333,18.267619,38.9422
2018-07-01,4.79,-50507.600000,1.953297e+06,27.087500,0.153000,62.950000,251.374500,205.094610,70.981429,241528.700000,...,128330.400000,0.282381,1.96,78.711136,0.382381,97.9,3.850000,1.282222,13.147619,45.0132
2018-10-01,5.07,-54487.555556,1.984727e+06,27.330000,-0.088205,62.866667,252.821833,205.510795,70.748696,244381.900000,...,128753.400000,0.291364,2.25,79.170891,0.215000,98.6,3.750000,1.428462,13.112500,51.953
2019-01-01,5.12,-50935.272727,2.014487e+06,27.575714,-0.171714,63.150000,252.568833,204.346439,51.396364,247500.700000,...,129247.333333,0.170476,2.37,79.020118,0.400000,91.2,4.000000,1.083077,19.572381,39.6474
2019-04-01,4.70,-50460.545455,2.033989e+06,27.780000,-0.611389,62.833333,255.139000,207.812833,63.770417,240404.888889,...,129681.666667,0.185417,2.38,78.033840,0.207000,97.2,3.650000,1.435714,13.302371,48.0063
2019-07-01,4.28,-51315.500000,2.045170e+06,28.017500,-0.330833,63.050000,255.926667,211.463051,57.358095,241007.111111,...,130005.888889,0.224091,2.10,77.643440,0.195000,98.4,3.666667,1.265714,13.890714,51.1618
2019-10-01,3.92,-44412.333333,2.043807e+06,28.220000,-0.622812,63.266667,257.265167,212.082206,53.963043,239093.333333,...,130536.875000,0.155455,1.65,77.057960,0.330000,95.5,3.600000,1.342500,17.481970,59.967


🛠️ Engineering features: lags and moving averages
✅ Feature engineering complete, resulting shape: (0, 150)


C:\Users\Steel\AppData\Local\Temp\ipykernel_7004\1699384153.py:62: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag{lag}'] = df[col].shift(lag)
C:\Users\Steel\AppData\Local\Temp\ipykernel_7004\1699384153.py:62: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag{lag}'] = df[col].shift(lag)
C:\Users\Steel\AppData\Local\Temp\ipykernel_7004\1699384153.py:62: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider

In [ ]:
def engineer_features(df: pd.DataFrame, target_col: str, lag_periods: List[int] = [1, 3, 6], ma_windows: List[int] = [3, 12]) -> pd.DataFrame:
    try:
        print("🛠️ Engineering features: lags and moving averages")
        df = df.copy()
        for col in df.columns:
            for lag in lag_periods:
                df[f'{col}_lag{lag}'] = df[col].shift(lag)
        for window in ma_windows:
            df[f'{target_col}_ma{window}'] = df[target_col].rolling(window=window).mean()
        df = df.dropna()
        print(f"✅ Feature engineering complete, resulting shape: {df.shape}")
        return df
    except Exception as e:
        print(f"❌ Error in feature engineering: {str(e)}")
        raise

In [15]:
df = combined
try:
    lag_periods: List[int] = [1, 3, 6]
    ma_windows: List[int] = [3, 12]
    print("🛠️ Engineering features: lags and moving averages")
    df = df.copy()

except Exception as e:
    print(f"❌ Error in feature engineering: {str(e)}")
    raise


🛠️ Engineering features: lags and moving averages


In [17]:
for col in df.columns:
    for lag in lag_periods:
        df[f'{col}_lag{lag}'] = df[col].shift(lag)

C:\Users\Steel\AppData\Local\Temp\ipykernel_7004\3590728991.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag{lag}'] = df[col].shift(lag)
C:\Users\Steel\AppData\Local\Temp\ipykernel_7004\3590728991.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag{lag}'] = df[col].shift(lag)
C:\Users\Steel\AppData\Local\Temp\ipykernel_7004\3590728991.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider jo

In [19]:
df.columns

Index(['BAA', 'BOPGSTB', 'BUSINV', 'CES0500000003', 'CFNAI', 'CIVPART',
       'CPIAUCSL', 'CSUSHPINSA', 'DCOILWTICO', 'DGORDER',
       ...
       'UNRATE_lag6', 'USSLIND_lag1', 'USSLIND_lag3', 'USSLIND_lag6',
       'VIXCLS_lag1', 'VIXCLS_lag3', 'VIXCLS_lag6', 'AAPL_Close_lag1',
       'AAPL_Close_lag3', 'AAPL_Close_lag6'],
      dtype='object', length=148)

In [21]:
target_col = f'AAPL_Close'
for window in ma_windows:
    df[f'{target_col}_ma{window}'] = df[target_col].rolling(window=window).mean()

C:\Users\Steel\AppData\Local\Temp\ipykernel_7004\645007122.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{target_col}_ma{window}'] = df[target_col].rolling(window=window).mean()
C:\Users\Steel\AppData\Local\Temp\ipykernel_7004\645007122.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{target_col}_ma{window}'] = df[target_col].rolling(window=window).mean()


In [23]:
df = df.fillna(method='bfill').fillna(method='ffill')

,BAA,BOPGSTB,BUSINV,CES0500000003,CFNAI,CIVPART,CPIAUCSL,CSUSHPINSA,DCOILWTICO,DGORDER,...,USSLIND_lag3,USSLIND_lag6,VIXCLS_lag1,VIXCLS_lag3,VIXCLS_lag6,AAPL_Close_lag1,AAPL_Close_lag3,AAPL_Close_lag6,AAPL_Close_ma3,AAPL_Close_ma12
2017-07-01,4.39,-43842.454545,1.878008e+06,26.358571,-0.142941,62.866667,244.254167,193.660364,46.600000,223990.454545,...,1.297000,1.297000,10.264500,10.264500,10.264500,34.6436,34.6436,34.6436,37.821967,NaN
2017-10-01,4.32,-45681.700000,1.892606e+06,26.497143,0.698529,62.700000,246.610333,195.238733,51.577727,233244.818182,...,1.297000,1.297000,10.264500,10.264500,10.264500,34.6436,34.6436,34.6436,37.821967,NaN
2018-01-01,4.26,-51366.000000,1.919995e+06,26.724000,-0.245758,62.700000,248.878000,196.254524,63.698571,233560.727273,...,1.297000,1.297000,10.125455,10.264500,10.264500,39.529,34.6436,34.6436,37.821967,NaN
2018-04-01,4.67,-46001.636364,1.935419e+06,26.890000,0.379211,62.850000,250.095667,200.750930,66.253810,245270.800000,...,1.297000,1.297000,11.062381,10.264500,10.264500,39.2933,34.6436,34.6436,39.254833,NaN
2018-07-01,4.79,-50507.600000,1.953297e+06,27.087500,0.153000,62.950000,251.374500,205.094610,70.981429,241528.700000,...,1.577826,1.297000,18.267619,10.125455,10.264500,38.9422,39.529,34.6436,41.082900,NaN
2018-10-01,5.07,-54487.555556,1.984727e+06,27.330000,-0.088205,62.866667,252.821833,205.510795,70.748696,244381.900000,...,1.554737,1.297000,13.147619,11.062381,10.264500,45.0132,39.2933,34.6436,45.302800,NaN
2019-01-01,5.12,-50935.272727,2.014487e+06,27.575714,-0.171714,63.150000,252.568833,204.346439,51.396364,247500.700000,...,1.568333,1.297000,13.112500,18.267619,10.264500,51.953,38.9422,34.6436,45.537867,NaN
2019-04-01,4.70,-50460.545455,2.033989e+06,27.780000,-0.611389,62.833333,255.139000,207.812833,63.770417,240404.888889,...,1.282222,1.577826,19.572381,13.147619,10.125455,39.6474,45.0132,39.529,46.535567,NaN
2019-07-01,4.28,-51315.500000,2.045170e+06,28.017500,-0.330833,63.050000,255.926667,211.463051,57.358095,241007.111111,...,1.428462,1.554737,13.302371,13.112500,11.062381,48.0063,51.953,39.2933,46.271833,NaN
2019-10-01,3.92,-44412.333333,2.043807e+06,28.220000,-0.622812,63.266667,257.265167,212.082206,53.963043,239093.333333,...,1.083077,1.568333,13.890714,19.572381,18.267619,51.1618,39.6474,38.9422,53.045033,NaN


In [ ]:
save_to_storage(combined_data, output_path, ticker, IS_DATABRICKS)